In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
#load previouisly exported CSV into a DataFrame.
core0 = pd.read_csv("../output_data/weather_metadata.csv")
core1 = core0.drop(columns=["Unnamed: 0"])
core1

,City,Country,Date,Latitude,Longitude,Maximum Temperature,Humidity,Cloud Level,Wind (m/h)
0,Bluff,NZ,1587583609,-46.60,168.33,48.99,85,82,1.99
1,Constitución,CL,1587583322,-35.33,-72.42,67.77,44,82,6.96
2,Kavieng,PG,1587583384,-2.57,150.80,81.25,76,100,0.72
3,Albany,US,1587583592,42.60,-73.97,45.00,28,75,34.45
4,Dingle,PH,1587583609,11.00,122.67,80.31,81,78,8.43
...,...,...,...,...,...,...,...,...,...
537,Kamakwie,SL,1587583656,9.50,-12.23,94.42,42,65,4.97
538,Palauig,PH,1587583656,15.44,119.90,82.51,77,87,4.07
539,Riyadh,SA,1587583656,24.69,46.72,78.80,57,30,6.93
540,Tubualá,PA,1587583656,9.52,-79.03,84.20,79,40,12.75


In [3]:
#Configuring gmaps, usuing latitude and longitudes as loncatrions and humidity as weight.
#adding the heatmap layer to initial map.
gmaps.configure(api_key=g_key)
humidity = core0["Humidity"]
location0 = core0[["Latitude","Longitude"]]
max0= float(humidity.max())
map0 = gmaps.figure()
heatmap0 = gmaps.heatmap_layer(location0, weights = humidity, dissipating=False,max_intensity = max0, 
                               point_radius=1)
map0.add_layer(heatmap0)
map0

Figure(layout=FigureLayout(height='420px'))

In [4]:
#Creating a new DataFrame according to given criteria.
core2 = core1.loc[(core1["Maximum Temperature"] >= 70) & (core1["Maximum Temperature"] < 80) & (core1["Wind (m/h)"] < 10) & (core1["Cloud Level"] ==0),:]
core3 = core2.dropna(how="any")
core3.reset_index(inplace=True)
core4 = core3.drop(columns=["index"])
core4

,City,Country,Date,Latitude,Longitude,Maximum Temperature,Humidity,Cloud Level,Wind (m/h)
0,Castro,BR,1587583614,-24.79,-50.01,73.72,31,0,4.83
1,Arraial do Cabo,BR,1587583623,-22.97,-42.03,78.80,69,0,9.17
2,Mount Isa,AU,1587583626,-20.73,139.50,77.00,17,0,8.05
3,Awjilah,LY,1587583633,29.11,21.29,79.07,17,0,7.81
4,Cananéia,BR,1587583651,-25.01,-47.93,75.20,64,0,6.55
5,Sinazongwe,ZM,1587583655,-17.26,27.46,72.84,44,0,3.11


In [5]:
#Creating a new dataframe for the suitable weather hotels and adding an empty column.
hotels0 = pd.DataFrame(core4,columns=["City","Country","Latitude", "Longitude"])
hotels0["Hotel Name"] = ""
hotels0

,City,Country,Latitude,Longitude,Hotel Name
0,Castro,BR,-24.79,-50.01,
1,Arraial do Cabo,BR,-22.97,-42.03,
2,Mount Isa,AU,-20.73,139.50,
3,Awjilah,LY,29.11,21.29,
4,Cananéia,BR,-25.01,-47.93,
5,Sinazongwe,ZM,-17.26,27.46,


In [6]:
#Calling the API and appending the an empty list to be used for the previously created empty column.
hotels1 = []
for i in range(len(core4)):
    latitude = core4.loc[i]["Latitude"]
    longitude = core4.loc[i]["Longitude"]
    parameters = {"location": f"{latitude},{longitude}","radius":5000, "types": "hotel","keyword":"hotel", "key" :g_key}
    url0 = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    request = requests.get(url0, params = parameters)
    response = request.json()
    try:
        hotels1.append(response["results"][0]["name"])
    except IndexError:
        hotels1.append("N/A - No Hotel Found")
hotels1

['Hotel Buganville Express',
 'Hotel da Canoa',
 'Abacus Motel',
 'N/A - No Hotel Found',
 'Hotel Pousada da Néia',
 'Lyawa Lodge']

In [7]:
#Adding the obtained results from the API to the hotels dataframe.
#Small mistake here as it would not return a hotel for the 4th location - Awjilah.
hotels0["Hotel Name"]= hotels1
hotels0

,City,Country,Latitude,Longitude,Hotel Name
0,Castro,BR,-24.79,-50.01,Hotel Buganville Express
1,Arraial do Cabo,BR,-22.97,-42.03,Hotel da Canoa
2,Mount Isa,AU,-20.73,139.50,Abacus Motel
3,Awjilah,LY,29.11,21.29,N/A - No Hotel Found
4,Cananéia,BR,-25.01,-47.93,Hotel Pousada da Néia
5,Sinazongwe,ZM,-17.26,27.46,Lyawa Lodge


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotels0.iterrows()]
locations = hotels0[["Latitude", "Longitude"]]

In [9]:
# Add marker layer ontop of heat map
marker0 = gmaps.marker_layer(locations)
map0.add_layer(marker0)
# Display Map
map0

Figure(layout=FigureLayout(height='420px'))